# ResNeXt50 from sratch
ref : https://towardsai.net/p/research/resnext-from-scratch


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

import torchvision
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)


Device cpu


In [2]:
FMnist = datasets.FashionMNIST(root="./FashionMnist", download=True)
train,test = train_test_split(FMnist, test_size=0.2, random_state=42, shuffle=True)

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./CIFAR', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,  # type: ignore
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./CIFAR', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, #type: ignore 
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torchvision.models as models

resnext50_32x4d = models.resnext50_32x4d()

#print (resnext50_32x4d)


In [9]:
myLoss = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnext50_32x4d.parameters(), lr=0.001, momentum=0.9)

In [12]:
def train_model(model, optimizer, data_loader, loss_module, num_epochs=100):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in tqdm(range(num_epochs)):
        for data_inputs, data_labels in data_loader:

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs = data_inputs.to(device)
            data_labels = data_labels.to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_labels)

            ## Step 4: Perform backpropagation
            # Before calculating the gradients, we need to ensure that they are all zero.
            # The gradients would not be overwritten, but actually added to the existing ones.
            optimizer.zero_grad()
            # Perform backpropagation
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()

In [13]:
train_model(resnext50_32x4d,optimizer,trainloader,myLoss)

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 